In [42]:
import os
import pandas as pd
from collections import Counter
import numpy as np

#supernoder files:

#вариант, который считает РАЗНЫЕ типы мотивов одного и того же размера
from SuperNoder_diff_types.manager import Manager as Manager_types 

#вариант, который считает все мотивы одного размера вместе
from SuperNoder.manager import Manager as Manager

**импортируем датасеты, и создаем каждому файлик с вершинами** 

In [23]:
path='Data'
datasets_names = ['\cora.txt','\email-univ.txt'] #правда надо, чтоб датасеты были одного вида
for dataset_name in datasets_names:
    name=dataset_name.split('.')
    dataset=pd.read_csv(path+dataset_name,sep=' ',header=None)
    if not os.path.exists(path+name[0]+'_nodes.txt'):
        col_1 = set(Counter(dataset[0].tolist()).keys())
        col_2=set(Counter(dataset[1].tolist()).keys())        
        nodes = sorted(col_1 | col_2)
        data_nodes = {'nodes':nodes,'labels': ['A']*len(nodes)}
        nodes_file=pd.DataFrame(data_nodes)
        nodes_file.to_csv(r'C:\Users\anpolol\Desktop\DifusionPredictionUsingMotifs\Data'+name[0]+'_nodes.txt', header=None, index=None, sep=' ', mode='a')

**arguments for SuperNoder:**

-n,  --nodes-file <filename> MANDATORY The list of nodes. Node id and label for each row separated by a space\
-e,  --edges-file <filename> MANDATORY The list of edges. One edge for each row.\
-m,  --method <method> OPTIONAL The heuristic to use in order to maximize motifs. DEFAULT: h1 \
-tn, --type-of-network <type> OPTIONAL 	The type of network. It can be chosen from [direct, undirect]. DEFAULT: undirect\ 
-th, --threshold <threshold> OPTIONAL The threshold to hold over-represented motifs.\
-ms, --motif-size <size> OPTIONAL The size of motifs. It must be greater or equal to 3. DEFAULT: 3\
-h1tr, --h1-times-repetition <times> OPTIONAL \tThe number of repetition of h1. DEFAULT: 1\
-ss, --samples-size <sample_size> OPTIONAL The size of samples for heuristics h4 and h5. DEFAULT: 100\

In [52]:
arg_tn='undirect'
arg_th = '1'
arg_ms = '3' 
arg_m = 'h1'
arg_ss = '100'
arg_h1tr = 1
arg_nr=1

distrubutions = dict()
for dataset_name in datasets_names:
    print('finding motifs for', dataset_name)
    name=dataset_name.split('.')
    arg_e = 'Data'+dataset_name
    arg_n = 'Data'+name[0]+'_nodes.txt'
    distrubutions[dataset_name] = {}
    for arg_ms in range(3,6):
        print('motif_size', arg_ms)
        argv=[' ','-e',arg_e, '-n',arg_n, '-tn',arg_tn,'-th',arg_th,'-ms',arg_ms,'-m',arg_m,\
      '-h1tr',arg_h1tr,'-ss',arg_ss]
        
        m = Manager_types(argv)
        distribution_disjoint, distribution_f1 = m.run() #первое значение словаря - тип мотива или размер мотива, второе значение - количество таких мотивов в графе
        
        distrubutions[dataset_name] = dict(list(distrubutions[dataset_name].items()) + list(distribution_disjoint.items()))

finding motifs for \cora.txt
motif_size 3

 Reading input
# 2021-02-18 00:28:44.330769 Loading graph
#	The original network has  5278  nodes and  5278  edges
# 2021-02-18 00:28:44.348772 Enumeration
#	The total number of motifs is  49041
# 2021-02-18 00:28:45.491775 Threshold computation
#	The number of motifs that occur more than 1 times is 2
# 2021-02-18 00:28:58.413066 Disjointness computation
#	The number of disjoint motifs is  2
SuperNoder execution has finished

motif_size 4

 Reading input
# 2021-02-18 00:28:58.505068 Loading graph
#	The original network has  5278  nodes and  5278  edges
# 2021-02-18 00:28:58.519085 Enumeration
#	The total number of motifs is  71383
# 2021-02-18 00:29:01.305085 Threshold computation
#	The number of motifs that occur more than 1 times is 6
# 2021-02-18 00:29:27.052152 Disjointness computation
#	The number of disjoint motifs is  6
SuperNoder execution has finished

motif_size 5

 Reading input
# 2021-02-18 00:29:27.164144 Loading graph
#	The origi

**составляем матрицу входных данных таким образом, чтоб она была одной и той же длины для любого датасета**

In [54]:
names_of_all_motifs=[] #list of all motif types
for dataset in distrubutions:
    for name_of_motif in distrubutions[dataset]:
        if name_of_motif not in names_of_all_motifs:
            names_of_all_motifs.append(str(name_of_motif))

In [56]:
X = np.zeros((len(datasets_names),len(names_of_all_motifs)))
for i,d in enumerate(datasets_names):
    for k,m in enumerate(names_of_all_motifs): #j - значение, i - номера
        if m in distrubutions[d]:
            X[i][k] = distrubutions[d][m]/sum(distrubutions[d].values())  